## Budgets for flux-form equations

The continuity equation
$$
\partial_t h_k = - \nabla \cdot \left( h_k {\bf v}_k \right)
$$
describes the evolution of the thickness of an ocean layer in a reduced gravity model. $k$ is the layer number in the vertical direction (usually $k=1$ is the top layer), $h_k$ is the layer thickness, $t$ is time, ${\bf v}_k = (u,v)$ is the horizontal velocity vector, and $\nabla \cdot \equiv (\partial_x, \partial_y)$ is the horizontal divergence operator where $x$ and $y$ are the horizontal spatial dimensions.

The divergence operator is discretized as
$$
\nabla \cdot \left( h_k {\bf v}_k \right) = \frac{1}{A}
\left( \delta_i \left( h_k u_k \right) + \delta_j \left( h_k v_k \right) \right) 
$$
where
$$
\delta_i \phi \equiv \phi_{i+\frac{1}{2},j} - \phi_{i-\frac{1}{2},j}
$$
and
$$
\delta_j \phi \equiv \phi_{i,j+\frac{1}{2}} - \phi_{i,j-\frac{1}{2}}
$$
and by convention $i$ is the cell number in the $x$-direction so that the center of the cell is $x_i =  ( i - \frac{1}{2} ) \Delta x$ for uniformly spaced grid. Similarly, $y_j = ( j - \frac{1}{2} ) \Delta y$.

Integrating the continuity equation over the time period between $t=t_A$ and $t=t_B$ we obtain
$$
\frac{ h_k(t_B) - h_k(t_A) }{ t_B - t_A } = - \nabla \cdot \left( \overline{ h_k {\bf v}_k } \right)
$$
where an overline indicates a time average:
$$
\overline{\phi} \equiv = \frac{ 1 }{ t_B - t_A } \int_{t_A}^{t_B} \phi(t) \,dt
$$

MOM6 can diagnose the time average of the flux terms in the continuity equations, $hu$ and $hv$, as well as the instantaneous $h$.

Verify that MOM6 diagnostics satisfy the continuity equation by
1. diagnose $\delta_i \left( h_k u_k \right) + \delta_j \left( h_k v_k \right)$ for levels $k=1$ and $k=2$
2. diagnose $A \left( h_k(t_B) - h_k(t_A) \right)$ for levels $k=1$ and $k=2$

In [19]:
%pylab inline
import netCDF4
import numpy as np

Populating the interactive namespace from numpy and matplotlib


In [2]:
from netCDF4 import Dataset
data = Dataset('200x200x5000days/snapshot.nc', "r", format="NetCDF4")

In [11]:
data['h']

<class 'netCDF4._netCDF4.Variable'>
float64 h(Time, zl, yh, xh)
    long_name: Layer Thickness
    units: m
    missing_value: 1e+20
    _FillValue: 1e+20
    cell_methods: area:mean zl:sum yh:mean xh:mean time: point
    cell_measures: area: areacello
unlimited dimensions: Time
current shape = (10, 2, 200, 200)
filling off

Noticed that `u` and `v` had an extra column/row.

In [12]:
data['u']

<class 'netCDF4._netCDF4.Variable'>
float32 u(Time, zl, yh, xq)
    long_name: Zonal velocity
    units: m s-1
    missing_value: 1e+20
    _FillValue: 1e+20
    cell_methods: zl:mean yh:mean xq:point time: point
    interp_method: none
unlimited dimensions: Time
current shape = (10, 2, 200, 201)
filling off

In [13]:
data['v']

<class 'netCDF4._netCDF4.Variable'>
float32 v(Time, zl, yq, xh)
    long_name: Meridional velocity
    units: m s-1
    missing_value: 1e+20
    _FillValue: 1e+20
    cell_methods: zl:mean yq:point xh:mean time: point
    interp_method: none
unlimited dimensions: Time
current shape = (10, 2, 201, 200)
filling off

Slicing each of the variables:

In [4]:
h = data['h'][0,0,:,:]
v = data['v'][0,0,:,:]
u = data['u'][0,0,:,:]

Instantiating a numpy array with zeroes that is 200 x 200

In [20]:
n = 200
arr = np.zeros((n,n), dtype=np.float)

Looping through `arr` to calculate the change in delta.

In [23]:
for j in range(n):
    for i in range(n):
        try:
            arr[j][i] = h[j,i] * u[j,i+1] - h[j,i] * u[j,i] + (h[j,i]*v[j+1,i] - h[j,i]*v[j,i])
        except:
            continue

/Users/Esther/miniconda2/envs/xarrays/lib/python3.6/site-packages/ipykernel/__main__.py:4: UserWarning: Warning: converting a masked element to nan.


Noticed that there were undefined values in the data that made for all the `nan`s that appeared when we tried to calculate.

In [25]:
arr

array([[        nan,         nan,         nan, ...,         nan,
                nan,         nan],
       [        nan, -0.10618791, -0.0556858 , ..., -0.00197806,
        -0.00331988,         nan],
       [        nan,  0.04338502,  0.03070331, ..., -0.00347992,
        -0.00473493,         nan],
       ...,
       [        nan,  0.01108602,  0.00743854, ..., -0.02460759,
        -0.0263961 ,         nan],
       [        nan,  0.0105397 ,  0.00648176, ..., -0.02889487,
        -0.03093488,         nan],
       [        nan,         nan,         nan, ...,         nan,
                nan,         nan]])